In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn import datasets

In [3]:
data = datasets.load_digits()

In [4]:
X = data.data

In [5]:
X[ :1, :]

array([[  0.,   0.,   5.,  13.,   9.,   1.,   0.,   0.,   0.,   0.,  13.,
         15.,  10.,  15.,   5.,   0.,   0.,   3.,  15.,   2.,   0.,  11.,
          8.,   0.,   0.,   4.,  12.,   0.,   0.,   8.,   8.,   0.,   0.,
          5.,   8.,   0.,   0.,   9.,   8.,   0.,   0.,   4.,  11.,   0.,
          1.,  12.,   7.,   0.,   0.,   2.,  14.,   5.,  10.,  12.,   0.,
          0.,   0.,   0.,   6.,  13.,  10.,   0.,   0.,   0.]])

In [12]:
y = data.target
y[:5]

array([0, 1, 2, 3, 4])

### Начнем с логистической регрессии LogisticRegression

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit

In [14]:
sss = StratifiedShuffleSplit(y)

In [75]:
#Создадим лист куда будем записывать модель и полученную точность
models_accuracy = {}

In [27]:
accuracies = []
lr = LogisticRegression(multi_class='multinomial',  penalty='l2', solver='lbfgs')
C_s = np.linspace(0.1, 2, 10)
for C in C_s:
    lr.C = C
    accuracies.append(np.mean(cross_val_score(lr, X, y, cv=sss, scoring='accuracy')))

In [47]:
for C, accuracy in zip(C_s, accuracies):
    print("Значение С = {}, точность = {}".format(round(C,2), round(accuracy, 3)))

Значение С = 0.1, точность = 0.969
Значение С = 0.31, точность = 0.965
Значение С = 0.52, точность = 0.967
Значение С = 0.73, точность = 0.972
Значение С = 0.94, точность = 0.97
Значение С = 1.16, точность = 0.96
Значение С = 1.37, точность = 0.963
Значение С = 1.58, точность = 0.967
Значение С = 1.79, точность = 0.962
Значение С = 2.0, точность = 0.966


In [58]:
lr.C = C_s[accuracies.index(max(accuracies))]

In [76]:
models_accuracy[lr] = max(accuracies)

Запишем нашу первую полученную модель и ее точность.

In [77]:
models_accuracy

{LogisticRegression(C=0.73333333333333328, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='multinomial', n_jobs=1, penalty='l2',
           random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
           warm_start=False): 0.97166666666666668}

### Переходим к рассмотрению решающего дерева DecisionTreeClassifier (код с занятия).

In [62]:
from sklearn.tree import DecisionTreeClassifier

In [68]:
means = []
depths = list(range(2, 20))
for depth in depths:
    clf = DecisionTreeClassifier(max_depth=depth)
    means.append(cross_val_score(estimator=clf, X=X, y=y, scoring='accuracy', cv=sss).mean())

In [69]:
for depth, mean in zip(depths, means):
    print("Глубина дерева = {}, точность = {}".format(round(depth,2), round(mean, 3)))

Глубина дерева = 2, точность = 0.319
Глубина дерева = 3, точность = 0.462
Глубина дерева = 4, точность = 0.567
Глубина дерева = 5, точность = 0.659
Глубина дерева = 6, точность = 0.755
Глубина дерева = 7, точность = 0.813
Глубина дерева = 8, точность = 0.834
Глубина дерева = 9, точность = 0.848
Глубина дерева = 10, точность = 0.867
Глубина дерева = 11, точность = 0.864
Глубина дерева = 12, точность = 0.861
Глубина дерева = 13, точность = 0.861
Глубина дерева = 14, точность = 0.852
Глубина дерева = 15, точность = 0.868
Глубина дерева = 16, точность = 0.867
Глубина дерева = 17, точность = 0.862
Глубина дерева = 18, точность = 0.859
Глубина дерева = 19, точность = 0.861


Наилучшие результаты показали деревья глубиной от 9 до 19. Самую большую точность имеет дерево глубиной - 12.

In [70]:
clf.max_depth = depths[means.index(max(means))]

In [78]:
models_accyracy[clf] = max(means)

In [79]:
models_accyracy

{LogisticRegression(C=0.73333333333333328, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='multinomial', n_jobs=1, penalty='l2',
           random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
           warm_start=False): 0.97166666666666668,
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=15,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'): 0.86777777777777787}

## Рассмотрим следующий алгоритм SVM.
Рассмотрим различные параметры C. Параметр С позволяет находить компромисс между максимизацией разделяющей полосы и минимизацией суммарной ошибки.

In [80]:
from sklearn import svm

In [93]:
svc = svm.SVC(kernel='linear')

In [105]:
C_s = np.logspace(-10, 0, 25)

In [106]:
import matplotlib.pyplot as plt

In [108]:
means = []
for C in C_s:
    svc.C = C
    means.append(np.mean(cross_val_score(svc, X, y, scoring='accuracy', cv=sss)))

In [111]:
for c, mean in zip(C_s, means):
    print("Параметр штрафа С = {}, точность = {}".format(round(c, 2), round(mean, 3)))

Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.1
Параметр штрафа С = 0.0, точность = 0.208
Параметр штрафа С = 0.0, точность = 0.802
Параметр штрафа С = 0.0, точность = 0.931
Параметр штрафа С = 0.0, точность = 0.952
Параметр штрафа С = 0.0, точность = 0.966
Параметр штрафа С = 0.0, точность = 0.975
Параметр штрафа С = 0.0, точность = 0.983
Параметр штрафа С = 0.0, точность = 0.981
Параметр штрафа С = 0.01, точность = 0.978
Параметр штрафа С = 0.02, точность = 0.979
Параметр штрафа С = 0.06, точность = 0.983
Параметр штрафа С = 0.15, точность = 0.981
Параметр штрафа С = 0.38, точность = 0.984
Параметр 

In [113]:
svc.C = C_s[means.index(max(means))]

In [114]:
models_accyracy[svc] = max(means)

In [115]:
models_accyracy

{LogisticRegression(C=0.73333333333333328, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='multinomial', n_jobs=1, penalty='l2',
           random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
           warm_start=False): 0.97166666666666668,
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=15,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'): 0.86777777777777787,
 SVC(C=0.38311868495572932, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False): 0.98388888888888881}

### Рассмотрим метод KNN (метод ближайших соседей).

In [161]:
from sklearn.neighbors import KNeighborsClassifier

In [162]:
knn = KNeighborsClassifier()
weights = ['uniform', 'distance']

In [163]:
means_uniform = []
#Рассмотрим сначала модель, где от расстояния не зависит вес
knn.weights = 'uniform'
for n_neighbors in list(range(2, 10)):
    knn.n_neighbors = n_neighbors
    means_uniform.append(np.mean(cross_val_score(knn, X, y, scoring='accuracy', cv=sss)))

In [164]:
means_uniform

[0.98888888888888893,
 0.98888888888888871,
 0.98666666666666669,
 0.98777777777777764,
 0.98999999999999999,
 0.98166666666666669,
 0.98722222222222222,
 0.98666666666666669]

In [165]:
means_distance = []
#Рассмотрим сначала модель, где от расстояния не зависит вес
knn.weights = 'distance'
for n_neighbors in list(range(2, 10)):
    knn.n_neighbors = n_neighbors
    means_distance.append(np.mean(cross_val_score(knn, X, y, scoring='accuracy', cv=sss)))

In [166]:
means_distance

[0.99388888888888904,
 0.98999999999999988,
 0.99055555555555552,
 0.98666666666666669,
 0.98611111111111105,
 0.98555555555555563,
 0.98999999999999999,
 0.98388888888888881]

In [167]:
max(means_uniform) > max(means_distance) #чуть лучше оказалась модель, где учитывется расстоние

False

In [168]:
knn.n_neighbors = list(range(2, 10))[means_distance.index(max(means_distance))]
knn.weights = 'distance'

In [172]:
models_accuracy[knn] = max(means_distance)

In [173]:
models_accuracy

{KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=1, n_neighbors=2, p=2,
            weights='distance'): 0.99388888888888904,
 LogisticRegression(C=0.73333333333333328, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='multinomial', n_jobs=1, penalty='l2',
           random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
           warm_start=False): 0.97166666666666668,
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=15,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'): 0.86777777777777787,
 SVC(C=0.38311868495572932, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degr

## Решающие деревья показали очень низкую точнсть, попробуем использовать ансамбли.

In [174]:
from sklearn.ensemble import RandomForestClassifier

In [197]:
rfc = RandomForestClassifier()

In [199]:
amoun_estimators = list(range(2, 100, 5))

In [200]:
means = []
for n_estimators in amoun_estimators:
    rfc.n_estimators = n_estimators
    means.append(np.mean(cross_val_score(rfc, X, y, scoring='accuracy', cv=sss)))

In [201]:
means

[0.78055555555555567,
 0.92388888888888887,
 0.95999999999999996,
 0.96444444444444444,
 0.96833333333333338,
 0.97055555555555539,
 0.9688888888888888,
 0.9722222222222221,
 0.97499999999999987,
 0.97444444444444434,
 0.97444444444444434,
 0.97333333333333327,
 0.96999999999999997,
 0.97777777777777786,
 0.9688888888888888,
 0.97777777777777763,
 0.97277777777777763,
 0.97333333333333338,
 0.97611111111111115,
 0.98333333333333339]

In [202]:
rfc.n_estimators = amoun_estimators[means.index(max(means))]

In [203]:
models_accyracy[rfc] = max(means)

In [204]:
models_accyracy

{KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
            metric_params=None, n_jobs=1, n_neighbors=2, p=2,
            weights='distance'): 0.99388888888888904,
 LogisticRegression(C=0.73333333333333328, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='multinomial', n_jobs=1, penalty='l2',
           random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
           warm_start=False): 0.97166666666666668,
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=15,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'): 0.86777777777777787,
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_fe

## Итоги. 
Самая худшая модель - DecisionTreeClassifier с точностью  = __0.877__, самая лучшая модель - KNeighborsClassifier с точностью = __0.994__. 
В KNeighborsClassifier используем следующие параметры: 
* количество соседей (__n_neighbors__) = 2
* веса (__weights__) = 'distance', то есть дальность напрямую влияет на расчет в модели